In [8]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Step 1: Load and Preprocess the Data
try:
    df = pd.read_excel('Untitled spreadsheet.xlsx')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: 'Untitled spreadsheet.xlsx' not found. Please ensure the file exists.")
    exit()
except Exception as e:
    print(f"An error occurred: {e}")
    exit()

# Remove irrelevant columns
irrelevant_cols = ['AID', 'Breathing', 'Awareness', 'Suitability']
df = df.drop(columns=irrelevant_cols, errors='ignore')

# Handle missing values
df['Contraindications'] = df['Contraindications'].fillna('none')
df['Targeted Mental Problems'] = df['Targeted Mental Problems'].fillna('none')
df['Targeted Physical Problems'] = df['Targeted Physical Problems'].fillna('none')

# Standardize text (lowercase for consistency)
for col in ['Level', 'Target Areas', 'Targeted Mental Problems', 'Targeted Physical Problems', 'Weight Goal Alignment']:
    df[col] = df[col].str.lower()

# Encode categorical variables
le_level = LabelEncoder()
df['Level'] = le_level.fit_transform(df['Level'])

le_target_areas = LabelEncoder()
df['Target Areas'] = le_target_areas.fit_transform(df['Target Areas'])

le_weight_goal = LabelEncoder()
df['Weight Goal Alignment'] = le_weight_goal.fit_transform(df['Weight Goal Alignment'])

# Multi-label binarize mental and physical problems
mlb_mental = MultiLabelBinarizer()
mental_problems = mlb_mental.fit_transform(df['Targeted Mental Problems'].str.split(', '))
mental_df = pd.DataFrame(mental_problems, columns=mlb_mental.classes_)

mlb_physical = MultiLabelBinarizer()
physical_problems = mlb_physical.fit_transform(df['Targeted Physical Problems'].str.split(', '))
physical_df = pd.DataFrame(physical_problems, columns=mlb_physical.classes_)

# Combine features into a single DataFrame
pose_features = pd.concat([df[['Level', 'Target Areas', 'Weight Goal Alignment']], mental_df, physical_df], axis=1)

def generate_training_data(df, pose_features, n_samples=1000):
    X, y = [], []
    all_mental_problems = list(mlb_mental.classes_)
    all_physical_problems = list(mlb_physical.classes_)
    target_areas_options = ['arms', 'legs', 'core', 'back', 'flexibility', 'balance']

    # Get encoded values for weight goals
    both_encoded = le_weight_goal.transform(['both'])[0] if 'both' in le_weight_goal.classes_ else None
    lose_encoded = [le_weight_goal.transform([wg])[0] for wg in le_weight_goal.classes_ if 'lose' in wg]
    gain_encoded = [le_weight_goal.transform([wg])[0] for wg in le_weight_goal.classes_ if 'gain' in wg]

    for _ in range(n_samples):
        # Simulate user profile
        height = np.random.randint(150, 200)
        weight = np.random.randint(50, 120)
        age = np.random.randint(20, 80)
        problem_type = np.random.choice(['mental', 'physical'])
        specific_problem = (np.random.choice(all_mental_problems) if problem_type == 'mental'
                           else np.random.choice(all_physical_problems))
        target_areas = np.random.choice(target_areas_options, size=np.random.randint(1, 4), replace=False)
        weight_goal = np.random.choice(['lose weight', 'gain muscle'])

        for idx, row in df.iterrows():
            # User features
            user_features = [
                height,
                weight,
                age,
                1 if problem_type == 'mental' else 0,
                1 if specific_problem in row['Targeted Mental Problems'].split(', ') or
                     specific_problem in row['Targeted Physical Problems'].split(', ') else 0,
                1 if 'arms' in target_areas else 0,
                1 if 'legs' in target_areas else 0,
                1 if 'core' in target_areas else 0,
                1 if 'back' in target_areas else 0,
                1 if 'flexibility' in target_areas else 0,
                1 if 'balance' in target_areas else 0,
                1 if weight_goal == 'lose weight' else 0
            ]

            # Pose features
            pose_vec = pose_features.iloc[idx].values

            # Label: 1 if suitable, 0 if not
            age_ok = age < 60
            weight_ok = weight < 100
            contra_ok = specific_problem not in row['Contraindications'].lower()
            problem_match = (problem_type == 'mental' and specific_problem in row['Targeted Mental Problems']) or \
                            (problem_type == 'physical' and specific_problem in row['Targeted Physical Problems'])

            # Target area match (assuming this is already fixed similarly)
            encoded_user_target_areas = [le_target_areas.transform([area])[0] for area in target_areas
                                        if area in le_target_areas.classes_]
            target_area_match = row['Target Areas'] in encoded_user_target_areas if encoded_user_target_areas else False

            # Fixed weight goal match
            pose_weight_goal = row['Weight Goal Alignment']
            if both_encoded is not None and pose_weight_goal == both_encoded:
                weight_goal_match = True
            elif weight_goal == 'lose weight' and any(pose_weight_goal == le for le in lose_encoded):
                weight_goal_match = True
            elif weight_goal == 'gain muscle' and any(pose_weight_goal == ge for ge in gain_encoded):
                weight_goal_match = True
            else:
                weight_goal_match = False

            label = 1 if age_ok and weight_ok and contra_ok and problem_match and target_area_match and weight_goal_match else 0

            X.append(np.concatenate([user_features, pose_vec]))
            y.append(label)

    return np.array(X), np.array(y)

# Call the function
X, y = generate_training_data(df, pose_features)
X, y = generate_training_data(df, pose_features)

# Step 3: Train the Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")




In [5]:
import joblib

# Save the trained model
joblib.dump(model, 'yoga_recommendation_model.pkl')

# Save all encoders
joblib.dump(le_level, 'le_level.pkl')
joblib.dump(le_target_areas, 'le_target_areas.pkl')
joblib.dump(le_weight_goal, 'le_weight_goal.pkl')
joblib.dump(mlb_mental, 'mlb_mental.pkl')
joblib.dump(mlb_physical, 'mlb_physical.pkl')

['mlb_physical.pkl']

In [6]:
# Load the saved model and encoders
model = joblib.load('yoga_recommendation_model.pkl')
le_level = joblib.load('le_level.pkl')
le_target_areas = joblib.load('le_target_areas.pkl')
le_weight_goal = joblib.load('le_weight_goal.pkl')
mlb_mental = joblib.load('mlb_mental.pkl')
mlb_physical = joblib.load('mlb_physical.pkl')

In [7]:
def recommend_poses(height, weight, age, target_areas_arms, target_areas_legs, target_areas_core,
                    target_areas_back, target_areas_flexibility, target_areas_balance,
                    weight_goal, health_type, specific_problem='none'):
    # Default specific problem if not provided
    if specific_problem == '':
        specific_problem = 'none'

    # Process target areas from Gradio checkboxes
    target_areas_list = []
    if target_areas_arms: target_areas_list.append('arms')
    if target_areas_legs: target_areas_list.append('legs')
    if target_areas_core: target_areas_list.append('core')
    if target_areas_back: target_areas_list.append('back')
    if target_areas_flexibility: target_areas_list.append('flexibility')
    if target_areas_balance: target_areas_list.append('balance')

    # Create user feature vector
    user_features = [
        height,
        weight,
        age,
        1 if health_type.lower() == 'mental' else 0,
        1 if specific_problem.lower() in (df['Targeted Mental Problems'] + df['Targeted Physical Problems']).str.lower() else 0,
        1 if 'arms' in target_areas_list else 0,
        1 if 'legs' in target_areas_list else 0,
        1 if 'core' in target_areas_list else 0,
        1 if 'back' in target_areas_list else 0,
        1 if 'flexibility' in target_areas_list else 0,
        1 if 'balance' in target_areas_list else 0,
        1 if weight_goal.lower() == 'lose weight' else 0
    ]

    # Predict suitability for all poses
    X_pred = []
    for idx in range(len(df)):
        pose_vec = pose_features.iloc[idx].values
        X_pred.append(np.concatenate([user_features, pose_vec]))

    probs = model.predict_proba(np.array(X_pred))[:, 1]  # Probability of suitability
    top_indices = np.argsort(probs)[::-1][:5]  # Top 5 recommendations

    # Format recommendations
    recommendations = df.iloc[top_indices][['AName', 'Description', 'Benefits', 'Contraindications', 'Level', 'Target Areas']]
    recommendations['Level'] = le_level.inverse_transform(recommendations['Level'])
    recommendations['Target Areas'] = le_target_areas.inverse_transform(recommendations['Target Areas'])
    return recommendations.to_html(index=False)

In [8]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Personalized Yoga Recommendations")

    with gr.Row():
        height = gr.Slider(100, 250, value=170, label="Height (cm)", step=1)
        weight = gr.Slider(30, 150, value=70, label="Weight (kg)", step=1)
        age = gr.Slider(10, 100, value=30, label="Age", step=1)

    gr.Markdown("### Target Areas")
    target_areas_arms = gr.Checkbox(label="Arms")
    target_areas_legs = gr.Checkbox(label="Legs")
    target_areas_core = gr.Checkbox(label="Core")
    target_areas_back = gr.Checkbox(label="Back")
    target_areas_flexibility = gr.Checkbox(label="Flexibility")
    target_areas_balance = gr.Checkbox(label="Balance")

    gr.Markdown("### Weight Goal")
    weight_goal = gr.Radio(choices=["Lose Weight", "Gain Muscle"], value="Lose Weight", label="Weight Goal")

    gr.Markdown("### Health Focus")
    health_type = gr.Radio(choices=["Mental", "Physical"], value="Physical", label="Mental or Physical Health")

    gr.Markdown("### Specific Problem (Optional)")
    specific_problem = gr.Textbox(label="Specific Problem (e.g., stress relief, high bp)", placeholder="Enter a specific problem or leave blank")

    submit_button = gr.Button("Get Recommendations")
    output = gr.HTML(label="Recommended Yoga Poses")

    submit_button.click(
        fn=recommend_poses,
        inputs=[height, weight, age,
                target_areas_arms, target_areas_legs, target_areas_core,
                target_areas_back, target_areas_flexibility, target_areas_balance,
                weight_goal, health_type, specific_problem],
        outputs=output
    )

# Launch the Gradio interface
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f939879f4840bcbf8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
# Save the dataset
df.to_excel('data.xlsx', index=False)

In [11]:
# When prompted for a password, use an access token with write permissions.
# Generate one from your settings: https://huggingface.co/settings/tokens
!git clone https://huggingface.co/spaces/ManasiPandit/Yoga_Recommender

Cloning into 'Yoga_Recommender'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (4/4), 1.29 KiB | 263.00 KiB/s, done.


In [12]:
!git add app.py
!git commit -m "Add application file"
!git push

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
